In [1]:
import os
import json
import argparse
import pprint as pp
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from datetime import timedelta

import networkx as nx
from scipy.spatial.distance import pdist, squareform
import torch
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from problems.tsp.problem_tsp import TSP
from utils import load_model, move_to
from train import set_decode_type

from torch.utils.tensorboard import SummaryWriter

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

In [3]:
class opts:
    dataset_path = "data/tsp/tsp10-200_concorde.txt"
    batch_size = 16
    accumulation_steps = 80
    num_samples = 25600 # 1280 samples per TSP size 
    
    neighbors = 0.20
    knn_strat = 'percentage'
    
    # model = "outputs/tsp_20-50/rl-ar-var-20pnn-gnn-max-ln_20200313T125908"
#     model = "outputs/tsp_20-50/rl-ar-var-20pnn-gnn-max_20200313T002243"
    
#     model = "outputs/tspsl_20-50/sl-ar-var-20pnn-gnn-sum_20200310T094801"
#     model = "outputs/tspsl_20-50/sl-ar-var-20pnn-gnn-max_20200308T172931"
#     model = "outputs/tspsl_20-50/sl-ar-var-20pnn-gnn-mean_20200310T094833"
#     model = "outputs/tspsl_20-50/sl-ar-var-full-mlp_20200306T182155"
    
#     model = "outputs/tspsl_20-50/sl-ar-var-20pnn-gnn-max-bntrack_20200310T095509"
#     model = "outputs/tspsl_20-50/sl-ar-var-20pnn-gnn-max-ln_20200310T095955"

    model = "outputs/tsp_20-50/rl-ar-var-20pnn-gnn-max-gaggr-sum_20200411T145725"
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

In [4]:
model, model_args = load_model(opts.model, extra_logging=True)
model.to(opts.device)
set_decode_type(model, "greedy")
model.eval()


Loading model from outputs/tsp_20-50/rl-ar-var-20pnn-gnn-max-gaggr-sum_20200411T145725/epoch-99.pt


AttentionModel(
  (init_embed): Linear(in_features=2, out_features=128, bias=True)
  (embedder): GNNEncoder(
    (init_embed_edges): Embedding(2, 128)
    (layers): ModuleList(
      (0): GNNLayer(
        (U): Linear(in_features=128, out_features=128, bias=True)
        (V): Linear(in_features=128, out_features=128, bias=True)
        (A): Linear(in_features=128, out_features=128, bias=True)
        (B): Linear(in_features=128, out_features=128, bias=True)
        (C): Linear(in_features=128, out_features=128, bias=True)
        (norm_h): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
        (norm_e): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      )
      (1): GNNLayer(
        (U): Linear(in_features=128, out_features=128, bias=True)
        (V): Linear(in_features=128, out_features=128, bias=True)
        (A): Linear(in_features=128, out_features=128, bias=True)
        (B): Linear(in_features=128, out_feat

In [5]:
tb_logger = SummaryWriter(os.path.join(
    model_args["log_dir"], "{}_{}-{}".format(model_args["problem"], model_args["min_size"], model_args["max_size"]), model_args["run_name"]))

In [6]:
dataset = TSP.make_dataset(
    filename=opts.dataset_path, batch_size=opts.batch_size, num_samples=opts.num_samples, 
    neighbors=opts.neighbors, knn_strat=opts.knn_strat, supervised=True
)
dataloader = DataLoader(dataset, batch_size=opts.batch_size, shuffle=False, num_workers=0)

  0%|          | 0/25600 [00:00<?, ?it/s]


Loading from data/tsp/tsp10-200_concorde.txt...


100%|##########| 25600/25600 [00:04<00:00, 5215.21it/s]


In [7]:
with torch.no_grad():
    graph_embs = None  # Store all graph embeddings for TB-projector 
    graph_embs_meta = None
    
    node_embs = None  # Reset after logging
    log_p = None
    log_p_selected = None
    
    for bat_idx, bat in enumerate(tqdm(dataloader, ascii=True)):
        x = move_to(bat['nodes'], opts.device)
        graph = move_to(bat['graph'], opts.device)
        cost, ll, pi = model(x, graph, return_pi=True)
        
        if node_embs is None:
            node_embs = model.embeddings_batch.cpu().numpy()
        else:
            # Append to node embeddings
            node_embs = np.concatenate((node_embs, model.embeddings_batch.cpu().numpy()), axis=0)
        
        if log_p is None:
            log_p = model.log_p_batch.cpu().numpy()
        else:
            # Append to log probabilities
            log_p = np.concatenate((log_p, model.log_p_batch.cpu().numpy()), axis=0)
        
        if log_p_selected is None:
            log_p_selected = model.log_p_sel_batch.cpu().numpy()
        else:
            # Append to log probabilities
            log_p_selected = np.concatenate((log_p_selected, model.log_p_sel_batch.cpu().numpy()), axis=0)
        
        if (bat_idx+1) % opts.accumulation_steps == 0:
            if graph_embs is None:
                graph_embs = node_embs.mean(1)
                graph_embs_meta = [f"TSP{10* ((bat_idx+1)//opts.accumulation_steps)}"]*len(node_embs)
            else:
                graph_embs = np.concatenate((graph_embs, node_embs.mean(1)), axis=0)
                graph_embs_meta += [f"TSP{10* ((bat_idx+1)//opts.accumulation_steps)}"]*len(node_embs)
            
            # Log prediction probabilities (for all action and selected actions)
            tb_logger.add_histogram('probs', np.exp(log_p.flatten()), 10* ((bat_idx+1)//opts.accumulation_steps))
            tb_logger.add_histogram('probs_selected', np.exp(log_p_selected.flatten()), 10* ((bat_idx+1)//opts.accumulation_steps))
            
            # Log histograms of raw values
            tb_logger.add_histogram('emb_values', node_embs.flatten(), 10* ((bat_idx+1)//opts.accumulation_steps))
            tb_logger.add_histogram('graph_emb_values', graph_embs.flatten(), 10* ((bat_idx+1)//opts.accumulation_steps))
            
            # Log histograms of L2 norms
            tb_logger.add_histogram('emb_2norm', np.linalg.norm(node_embs, axis=-1).flatten(), 10* ((bat_idx+1)//opts.accumulation_steps))
            tb_logger.add_histogram('graph_emb_2norm', np.linalg.norm(graph_embs, axis=-1).flatten(), 10* ((bat_idx+1)//opts.accumulation_steps))
            
            # Log histogram of distances between node embeddings (within each graph)
            node_embs_dists = []
            for node_emb in node_embs:
                # compute pdist for node embeddings within each graph
                node_embs_dists.append(pdist(node_emb, metric='euclidean'))
            tb_logger.add_histogram('emb_dist', np.array(node_embs_dists).flatten(), 10* ((bat_idx+1)//opts.accumulation_steps))
            
            # Log histogram of distances between graph embeddings
            graph_embs_dists = pdist(graph_embs, metric='euclidean')
            tb_logger.add_histogram('graph_emb_dist', np.array(graph_embs_dists).flatten(), 10* ((bat_idx+1)//opts.accumulation_steps))
            
            node_embs = None
            log_p = None
            log_p_selected = None
    
    # Log graph embeddings to projector
    tb_logger.add_embedding(graph_embs, metadata=graph_embs_meta, tag='graph_emb')

100%|##########| 1600/1600 [15:55<00:00,  1.67it/s]  
